# What is TSCC?
- The [Triton Shared Computing Cluster (TSCC)](https://www.sdsc.edu/support/user_guides/tscc-quick-start.html) is the cluster that we (Yeo lab, others at UCSD) use for processing and analysis of single-cell RNASeq data. It is a part of the [San Diego Supercomputer Center](https://www.sdsc.edu/) which has an administrative team that helps us create/restore backups, install top-level updates, and manage the job submission software for scripts that can be distributed across multiple machines (nodes). 
- Having an external administrative team is useful for reducing the engineering overhead, however this also means we lack administrative (superuser, or "[sudo](https://en.wikipedia.org/wiki/Sudo)") privileges. <b>This is common among labs within large organizations and Universities</b>. Software that requires these privileges needs either a workaround or administrator help. 

# Terminology
- **Unix**: Unix is a family of operating systems (think Windows or MacOS). Linux is a free* version of Unix, which has become the core for many distributions such as Centos, Ubuntu, and Fedora, each which specializes in different things. Since we're mostly going to be using Python and R (*scripting or interpreted language* as opposed to *compiled languages* like C or Java), it doesn't really matter which Unix distribution we're using so as long as the Python or R interpreter is installed. MacOSX was designed with Unix principles in mind, making it partially compatible with Unix and a popular choice among bioinformaticians. 
- **HPC**: High-Performance Computing (HPC) simply describes a computing environment specialized for high throughput or computationally demanding analysis. HPC clusters usually require additional software to manage job (script) submission and resources within and across its network of nodes (computers). TSCC uses the [TORQUE resource manager](https://en.wikipedia.org/wiki/TORQUE), so we will be running jobs by wrapping them inside TORQUE scripts (qscripts) and submitting them to the TORQUE resource manager. **Note**: Other HPC clusters may use other resource managers (SLURM is another popular one) and therefore will have different syntax, but the idea is the same.
- **Anaconda**: Anaconda is an open-source distribution (collection of softwares) for scientific computing. It contains the Python interpreter as well as a bunch of other useful software packages (such as Scipy, Jupyter, Numpy, etc.), and conda. 
- **Conda**: conda is a package manager, or software that helps organize these packages, plus any dependencies, versions, metadata that comes with it.
- **Local/remote**: "Local" usually refers to your own laptop/desktop, while "remote" will typically refer to the cluster or cloud instance that you are connecting to. You will almost always have a separate set of credentials (username/password) for each (for example, my local username is "brianyee" while my username on TSCC may be something like "ucsd-train03").
- **Node (Cluster)**: A single node is one server/machine/computer. A cluster is comprised of a group of nodes. Each node that we have access to provides up to 16 cores (processors) and up to 126Gb of memory, managed by the job scheduler (Maui) and the resource manager (TORQUE). 

# Scratch

Every user is allocated 100GB inside their home directory, which is typically used for permanent storage of processing scripts, notebooks, manuscript figures, etc. However this is not nearly enough to process large datasets, where intermediate files alone may take up several TB of space. Fortunately, TSCC provides a separate storage allocation specifically for temporary storage called *scratch*. For our purposes, consider this unlimited storage where we will do most of our processing. Let's make things easier by **softlinking** the path to scratch onto our home:

```ln -s /oasis/tscc/scratch/ucsd-trainXY ~/scratch```

**Note**: The ```~``` is another way of specifying your home directory, therefore typing ```~``` is identical to typing ```/home/ucsd-trainXY```.

# Screens (important!)

Each time you open a terminal and login to TSCC, you're starting a new session. However, since these sessions are not persistent, *closing your terminal window or logging out will also kill/stop any currently running job or command.* Obviously this doesn't work for long-running jobs, so we will be using screens to keep your sessions running even after you've closed your terminal. 

### Using a screen, you can keep a job running, even after you close the window! 

By default, screens are kind of ugly and not so informative, so we will be downloading and using a custom config (.screenrc) file that Olga (previous instructor) created. After you've logged into TSCC, download the config file:

```bash
cd ~ # change directory to your HOME
wget https://raw.githubusercontent.com/byee4/rcfiles/master/.screenrc
```

**Log out and log back in**, then type ```screen```

This .screenrc adds a status bar at the bottom of your screen, like this:

![screen example](img/screen.png)

### Common screen commands:

**Note**: By default, screen commands use a "control character" ```a``` to prefix a "screen" command, however this configuration modifies the control character to be ```j``` instead. We do this because ```Ctrl-a``` is itself another command that you can use to move your cursor all the way to the beginning of the commandline. Just a personal preference.

- **```screen```** : starts a new screen session
- **```Ctrl-j, c```** : creates a new tab within a screen session
- **```Ctrl-j, k```** : kills a screen tab 
- **```Ctrl-j, n```** and **```Ctrl-j, p```** : activates/toggles the "next" and "previous" tabs, respectively.
- **```Ctrl-j, #```** : activates the numbered tab within a screen session
- **```Ctrl-j, Shift-a```** : allows you to re-name a screen tab.

### Attaching/detaching screen sessions:

**Note**: When you are inside an active screen, simply closing the terminal window (or shutting off your computer) with an active screen session will simply "detach" it from your terminal, meaning the screen (and all the programs within the session) will continue to run.
- **```screen -r```** : Re-attach a screen (when you want to login and go back to an active screen)
- **```screen -d -r #####```** : De-tach and re-attach a screen (should you have multiple screen sessions, you can decide which one to attach). The ```#####``` represents the screen ID (by default, screens are assigned a numeric ID which become apparent when you use multiple screens).

### **Note**: 
Last year, several students ran into issues because they accidentally started several screen sessions (sometimes screen sessions within a screen session). This is okay! Simply re-attach a defunct screen ```screen -d -r #####``` and ```Ctrl-j, k``` to kill tabs until the screen session ends. To avoid this, I recommend always typing ```screen -d -r``` which will either: 

1. re-attach an existing screen session, or 
2. complain that you don't have any active screen sessions. (Then you're free to ```screen```)

# Working with TSCC

As said before, TSCC is a cluster of computers managed by the **TORQUE** resource manager, which we will need to run requests or reserve resources through. The following job submission commands are specific to **TORQUE** resource managers only, they will only work on HPC systems that use the **TORQUE** resource manager. 


#### When you login, you will be placed inside one of two login nodes (also called head nodes), which you can usually tell through the command prompt: 

![login1 node](img/login1.png)

![login2 node](img/login2.png)

Head nodes serve as a default, but as they are unreserved resources, we recommend **not using these nodes to run jobs!** The world won't end, but you'll probably run out of memory quickly and your processes will probably die. Instead, we'll request resources from compute nodes.

### There are two ways to request the resources:

# Submitting a job: 

Use a submission script to run specific command-line tools (ie. STAR, bowtie). These jobs will use the requested resources to simply run the commands inside the submission script on a compute node, whenever that compute node becomes available. Walltime (requested time to allocate to your command) will end as soon as the command finishes, so feel free to be liberal (I usually request double the amount of time I think I need per job to ensure that I won't run out of walltime). This is the preferred method for running software normally, however we will be running commands individually and therefore need to interact with the terminal console, so we will be submitting interactive jobs (below).

Here is an example submission script to request *non-interactive* resources for a job. Feel free to copy/look at it:

```
/oasis/tscc/scratch/CSHL_single_cell_2019/programming_exercise_material/example_qsub_job.sh
```

# Submitting an interactive job: 

Instead of supplying a submission script, you may include the ```-I``` parameter, which means that when the request is granted, you will be redirected to a bash shell within the compute node, leaving you free to run any commands inside the shell until walltime is exhausted. This is different from the above job in that instead of running a predetermined command, you are placed inside the compute node to run your own commands. Since your interactive node will last the entire walltime, please be considerate and not reserve resources for longer than you need (below is a typical request for an appropriate amoung of resources for the work we will be doing) **This is preferred for piloting commands or running software that requires you to interact with it**.

Here is an example interactive command (requesting a 1-node, 1 processor-per-node ```nodes=1:ppn=1``` interactive session ```-I``` lasting 6 hours ```walltime=6:00:00``` using the Yeo lab queue ```-q home-yeo```. We are leaving the memory requirements up to Maui/Torque:
```bash
qsub -I -l walltime=6:00:00 -q home-yeo -l nodes=1:ppn=1
```

Upon reservation, your prompt will indicate the node you're currently logged in as (for example, node tscc-1-44):

![tscc144 node](img/tscc-1-44.png)


# Conda/Modules

Our lab primarily uses conda to manage user-level packages (software installed for an individual user), and conda+modules to manage system and lab-wide installations. Modules basically help us point to where the lab-wide software is, and may be a feature in other scientific computing clusters. We'll be covering conda in another notebook, but for this course we'll be using software that has already been made available lab-wide. So there shouldn't be a need to install anything individually. 

### Common module commands:

- **```module avail```** : list all available modules
- **```module load samtools/1.5```** : loads samtools 1.5 (samtools/1.5 can be replaced with any module/version available). Not specifying a version will cause the default module to load.
- **```module list```** : lists all active modules, in the order they were loaded.
- **```module unload samtools/1.5```** : unloads the samtools module. Generally it is good practice to unload modules after use, since they modify environment variables such as your ```$PATH```

## Try loading and running the samtools module:

```samtools``` (you should get: command not found)

```module load samtools/1.5```

```samtools --version``` (you should get: 1.5)

## We will be using the following modules for this course:

Try getting an interactive node and loading these modules:

- seurat/3.0
- scanpy/1.4.1
- cellranger/3.0.2
- dropseqtools/1.13


# **Exercise 1: Use the cellranger module to get reads from a published dataset on GEO for re-processing**
- In several cases, BAM files were stored on GEO in lieu of fastq files, however the Cellranger pipeline will only accept sequencing data in the form of fastqs. 
- Let's use a dataset as an example from this paper: [A single-cell survey of the small intestinal epithelium](https://www.nature.com/articles/nature24489)
- [GSM2839445](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2839445)
- Load the cellranger module and see if you can use their software to convert a [BAM file from GEO](https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR6254382).
- You may find their online documentation useful. Google is your friend!
- **Use a screen!**


# TL;DR

```bash
ssh ucsd-trainXY@tscc-login1.sdsc.edu
cd ~
wget https://raw.githubusercontent.com/olgabot/rcfiles/master/.screenrc
screen -d -r # OR screen if you don't already have an active session yet
ln -s /oasis/tscc/scratch/ucsd-trainXY ./scratch
qsub -I -l walltime=6:00:00 -q home-yeo -l nodes=1:ppn=1
module load cellranger
cellranger
```